In [44]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt

In [45]:
# Načtení datasetu
df = pd.read_csv("../data/AirQualityUCI.csv", sep=';', decimal=',')

# Odstranění prázdných sloupců
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Odstranění prázdných řádků
df.dropna(how='all', inplace=True)

In [46]:
# Odstranění všech záznamů, které obsahují chybějící hodnoty (-200) v některém z atributů
cols_to_check = ['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)',
                 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
                 'T', 'RH', 'AH']

df1 = df[(df[cols_to_check] != -200).all(axis=1)]

In [47]:
df1.describe().round(2)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00,827.00
mean,2.35,1207.88,231.03,10.77,966.12,143.50,963.30,100.26,1600.62,1045.81,15.60,49.05,0.83
std,1.41,241.82,208.46,7.42,266.42,81.83,265.91,31.49,302.29,400.13,4.83,15.27,0.18
min,0.30,753.00,7.00,0.50,448.00,12.00,461.00,19.00,955.00,263.00,6.30,14.90,0.40
25%,1.30,1017.00,77.00,4.80,754.00,81.00,769.00,78.50,1369.50,760.00,11.90,36.70,0.72
50%,2.00,1172.00,157.00,9.10,944.00,128.00,920.00,99.00,1556.00,1009.00,15.00,49.60,0.82
75%,3.10,1380.00,318.50,14.80,1142.50,187.00,1131.00,122.00,1783.50,1320.00,18.30,60.55,0.93
max,8.10,2040.00,1189.00,39.20,1754.00,478.00,1935.00,196.00,2679.00,2359.00,30.00,83.20,1.49


In [48]:
pd.options.mode.chained_assignment=None

In [49]:
# Původní distribuce teploty
original_t = df1['T'].values

In [50]:
# Vytvoření nového atributu pro teplotu s mírným driftem
df1['T_drifted'] = df1['T'] + np.random.normal(loc=1.0, scale=0.5, size=len(original_t))

df1['T_drifted'] = drifted_t

In [51]:
df1.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,T_drifted
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,14.921851
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,14.082164
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,13.161897
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,11.655902
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,12.513613


### Kolmogorov Smirnov test

In [52]:
# KS test vrací statistiku a p-hodnotu
ks_stat, p_value = ks_2samp(original_t, drifted_t)

print(f"KS statistika: {ks_stat:.4f}")
print(f"p-hodnota: {p_value:.4f}")

if p_value < 0.05:
    print("Detekován drift (statisticky významný rozdíl mezi distribucemi)")
else:
    print("Drift nebyl detekován")


KS statistika: 0.1088
p-hodnota: 0.0001
Detekován drift (statisticky významný rozdíl mezi distribucemi)


### PSI